In [1]:
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import (
    BrokenPowerLaw,
    EblAbsorptionModel,
    ExponentialCutoffPowerLaw,
    ExponentialCutoffBrokenPowerLaw,
    InverseCompton,
    Synchrotron,
)

## Read data

soft_xray1 = ascii.read("mos1+mos2_pwn_nh-corr.dat")
#soft_xray2 = ascii.read("chandra_pwn_nh-corr.dat")
vhe = ascii.read("HAWC_J1826-128.dat")

## Model definition

def ElectronSynIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = 10 ** pars[0] / u.eV
    e_break = (10 ** pars[1]) * u.TeV
    alpha1 = pars[2]
    alpha2 = pars[3]
    e_cutoff = (10 ** pars[4]) * u.TeV
    B = pars[5] * u.uG

    # Initialize instances of the particle distribution and radiative models
    ECBPL = ExponentialCutoffBrokenPowerLaw(amplitude, 1.0 * u.TeV, e_break, alpha1, alpha2, e_cutoff, beta=1.0)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECBPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 9.5 * u.K, 10.0 * u.eV / u.cm ** 3],
        ],
        Eemin=1 * u.GeV,
    )
    SYN = Synchrotron(ECBPL, B=B)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=3.5 * u.kpc) + SYN.flux(
        data, distance=3.5 * u.kpc
    )

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stored in the sampler metadata
    # blobs.
    return model, IC.compute_We(Eemin=1 * u.TeV)


## Prior definition


def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """
    # Limit norm and B to be positive
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
        + naima.uniform_prior(pars[3], 0, np.inf)
    )

    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
    B0 = 2 * naima.estimate_B(soft_xray1, vhe).to("uG").value
    #B0 = 5

    p0 = np.array((33, 1, 1.7, 2.8, np.log10(48.0), B0))
    labels = ["log10(norm)", "e_break", "alpha1", "alpha2", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray1, vhe],
        p0=p0,
        labels=labels,
        model=ElectronSynIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=False,
    )

#    imf = naima.InteractiveModelFitter(
#        ElectronSynIC,
#        p0,
#        labels=labels,
#        e_range=[1e-09 * u.GeV, 1e05 * u.GeV],
#    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("J1826_HAWC_SynIC", sampler)
    
    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "J1826_HAWC_ECBPL",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("J1826_HAWC_ECBPL", sampler)
    
    plt = naima.plot_fit(sampler, label="Eel Pulsar Wind Nebula SED", sed=True, confs=[3,1], e_range=[0.00001*u.eV, 100*u.TeV])
    
    plt.savefig('J1826_SED_ECBPL.pdf', format='pdf', bbox_inches='tight')
    
    plt.show()

INFO: Finding Maximum Likelihood parameters through Nelder-Mead fitting... [naima.core]
INFO:    Initial parameters: [35.77545396  1.          1.7         2.8         1.68124124  1.33046388] [naima.core]
INFO:    Initial lnprob(p0): -1345424.572 [naima.core]
INFO:    New ML parameters : [32.37669356  0.48875075  1.5187479   1.87316026  2.97375897  2.18302464] [naima.core]
INFO:    Maximum lnprob(p0): -14.951 [naima.core]
Burning in the 32 walkers with 100 steps...

Progress of the run: 0 percent (0 of 100 steps)
                           --log10(norm)-- ----e_break---- ----alpha1----- ----alpha2----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.4            0.488           1.52            1.87            2.97            2.18      
  Last ensemble std    :      0.107          0.0038          0.00647         0.0117           0.015          0.0166     
  Last ensemble lnprob :  avg: -28.163, max: -14.781

Progress of the run: 5 percent (5 of 100 steps)
               


Progress of the run: 85 percent (85 of 100 steps)
                           --log10(norm)-- ----e_break---- ----alpha1----- ----alpha2----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.7            0.482           1.84            2.07            3.16             2.7      
  Last ensemble std    :      0.656           0.49            0.587           0.359           1.33            0.365     
  Last ensemble lnprob :  avg: -12.619, max: -9.511

Progress of the run: 90 percent (90 of 100 steps)
                           --log10(norm)-- ----e_break---- ----alpha1----- ----alpha2----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.7            0.475           1.84            2.05            3.15            2.73      
  Last ensemble std    :      0.672           0.517           0.656           0.357           1.45            0.391     
  Last ensemble lnprob :  avg: -12.669, max: -9.450

Progress of the run: 95 percent (95 of 100 steps)
             


Progress of the run: 75 percent (15 of 20 steps)
                           --log10(norm)-- ----e_break---- ----alpha1----- ----alpha2----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.9            0.467           2.01             2.1            3.24            2.79      
  Last ensemble std    :      0.648           0.584           0.825           0.288           1.49            0.516     
  Last ensemble lnprob :  avg: -12.347, max: -9.533

Progress of the run: 80 percent (16 of 20 steps)
                           --log10(norm)-- ----e_break---- ----alpha1----- ----alpha2----- -log10(cutoff)- -------B-------
  Last ensemble median :      32.9            0.467           2.01             2.1            3.24            2.79      
  Last ensemble std    :      0.667           0.589           0.844           0.293            1.5            0.544     
  Last ensemble lnprob :  avg: -12.182, max: -9.533

Progress of the run: 85 percent (17 of 20 steps)
                

INFO: Plotting chain of parameter log10(norm)... [naima.analysis]
INFO: -------------------log10(norm)--------------------
          log10(norm) = $32.8^{+0.6}_{-0.9}$
                 norm = $(6^{+17}_{-6})\times10^{32}$ [naima.plot]
INFO: Plotting chain of parameter e_break... [naima.analysis]
INFO: ---------------------e_break----------------------
          e_break = $0.5 \pm 0.4$ [naima.plot]
INFO: Plotting chain of parameter alpha1... [naima.analysis]
INFO: ----------------------alpha1----------------------
          alpha1 = $2.0^{+0.5}_{-0.8}$ [naima.plot]
INFO: Plotting chain of parameter alpha2... [naima.analysis]
INFO: ----------------------alpha2----------------------
          alpha2 = $2.1 \pm 0.3$ [naima.plot]
INFO: Plotting chain of parameter log10(cutoff)... [naima.analysis]
INFO: ------------------log10(cutoff)-------------------
          log10(cutoff) = $3.2^{+2}_{-0.9}$
                 cutoff = $(1.6^{+200}_{-1.4})\times10^{3}$ [naima.plot]
INFO: Plotting chain of

/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  f.clf()
/Users/danielburgess/anaconda3/lib/python3.7/site-packages/naima/analysis.py:160: UserWarning: Attempting to set identical left == right == 1.0 results in singular transformations; automatically expanding.
  f.clf()


INFO: Plotting $W_e$($E_e>1$ TeV)... [naima.analysis]
INFO: Saving results table in J1826_HAWC_ECBPL_results.ecsv [naima.analysis]


/Users/danielburgess/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:124: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.


In [ ]:
import astropy.units as u
import numpy as np
from astropy.io import ascii

import naima
from naima.models import (
    BrokenPowerLaw,
    EblAbsorptionModel,
    ExponentialCutoffPowerLaw,
    ExponentialCutoffBrokenPowerLaw,
    InverseCompton,
    Synchrotron,
)

## Read data

soft_xray1 = ascii.read("mos1+mos2_pwn_nh-corr.dat")
#soft_xray2 = ascii.read("chandra_pwn_nh-corr.dat")
vhe = ascii.read("HAWC_J1826-128.dat")

## Model definition

def ElectronSynIC(pars, data):

    # Match parameters to ECPL properties, and give them the appropriate units
    amplitude = 10 ** pars[0] / u.eV
    e_break = (10 ** pars[1]) * u.TeV
    alpha1 = pars[2]
    alpha2 = pars[3]
    e_cutoff = (10 ** pars[4]) * u.TeV
    B = pars[5] * u.uG

    # Initialize instances of the particle distribution and radiative models
    ECBPL = ExponentialCutoffBrokenPowerLaw(amplitude, 1.0 * u.TeV, e_break, alpha1, alpha2, e_cutoff, beta=2.0)
    # Compute IC on CMB and on a FIR component with values from GALPROP for the
    # position of RXJ1713
    IC = InverseCompton(
        ECBPL,
        seed_photon_fields=[
            "CMB",
            ["FIR", 9.5 * u.K, 0.5 * u.eV / u.cm ** 3],
        ],
        Eemin=100 * u.GeV,
    )
    SYN = Synchrotron(ECBPL, B=B)

    # compute flux at the energies given in data['energy']
    model = IC.flux(data, distance=3.5 * u.kpc) + SYN.flux(
        data, distance=3.5 * u.kpc
    )

    # The first array returned will be compared to the observed spectrum for
    # fitting. All subsequent objects will be stored in the sampler metadata
    # blobs.
    return model, IC.compute_We(Eemin=1 * u.TeV)


## Prior definition


def lnprior(pars):
    """
    Return probability of parameter values according to prior knowledge.
    Parameter limits should be done here through uniform prior ditributions
    """
    # Limit norm and B to be positive
    logprob = (
        naima.uniform_prior(pars[0], 0.0, np.inf)
        + naima.uniform_prior(pars[1], -1, 5)
        + naima.uniform_prior(pars[3], 0, np.inf)
    )

    return logprob


if __name__ == "__main__":

    ## Set initial parameters and labels
    # Estimate initial magnetic field and get value in uG
    B0 = 2 * naima.estimate_B(soft_xray1, vhe).to("uG").value
    #B0 = 5

    p0 = np.array((33, 1, 1.7, 2.8, np.log10(48.0), B0))
    labels = ["log10(norm)", "e_break", "alpha1", "alpha2", "log10(cutoff)", "B"]

    ## Run sampler
    sampler, pos = naima.run_sampler(
        data_table=[soft_xray1, vhe],
        p0=p0,
        labels=labels,
        model=ElectronSynIC,
        prior=lnprior,
        nwalkers=32,
        nburn=100,
        nrun=20,
        threads=4,
        prefit=True,
        interactive=True,
    )

#    imf = naima.InteractiveModelFitter(
#        ElectronSynIC,
#        p0,
#        labels=labels,
#        e_range=[1e-09 * u.GeV, 1e05 * u.GeV],
#    )

    ## Save run results to HDF5 file (can be read later with naima.read_run)
    naima.save_run("J1826_HAWC_SynIC", sampler)
    
    ## Diagnostic plots
    naima.save_diagnostic_plots(
        "J1826_HAWC_ECBPL",
        sampler,
        sed=True,
        blob_labels=["Spectrum", "$W_e$($E_e>1$ TeV)"],
    )
    naima.save_results_table("J1826_HAWC_ECBPL", sampler)
    
    plt = naima.plot_fit(sampler, label="Eel Pulsar Wind Nebula SED", sed=True, confs=[3,1], e_range=[0.00001*u.eV, 100*u.TeV])
    
    plt.savefig('J1826_SED_ECBPL.pdf', format='pdf', bbox_inches='tight')
    
    plt.show()